#### Clasificación neuronas actor

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import sys


# --- Add Modules directory to Python path ---
module_path = os.path.abspath(os.path.join('.', 'Modules'))

if module_path not in sys.path:
    sys.path.append(module_path)
    print(f"Added '{module_path}' to sys.path")
else:
    print(f"'{module_path}' already in sys.path")

Added '/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/Segundo cuatrimestre/TFM/Código/Modules' to sys.path


In [11]:
def classify_neurons_by_last_valid(
    FR_tensor: np.ndarray,
    measurements: list,
    window_size: int,
    R_A: float,
    R_B: float,
    p_global_thresh: float = 0.01,
    p_individual_thresh: float = 0.05
) -> list:
    """
    Clasifica cada neurona usando solamente aquellos trials en los que
    `measurements[t]` no es None (es decir, trials exitosos con recompensa ≥ R_B).
    Para cada neurona m y cada trial válido t, toma la media de los últimos
    `window_size` timesteps no-NaN de FR_tensor[m, :, t] como su FR_m(t).

    Luego ajusta para cada neurona m la regresión:

      FR_m(t) = β0
             + β1·Adiff
             + β2·AVR
             + β3·AVL
             + β4·(CVR − CVL)
             + β5·(CVR + CVL)
             + β6·Jpos
             + ε

    y, si el F-test global del modelo tiene p < p_global_thresh, clasifica a
    la neurona como “value-encoding”, determinando a continuación el coeficiente
    (β₁..β₆) con p-valor más bajo para asignar su sub-tipo. Si p ≥ p_global_thresh,
    la clasifica como “non-selective”.

    Parámetros:
    -----------
    FR_tensor : np.ndarray, shape = (n_neurons, n_timesteps, n_trials)
        Firing‐rates de cada neurona en cada timestep de cada trial. Los timesteps
        “extra” tras la finalización de un trial están rellenados con NaN para todos 
        los canales (ver cómo lo construye el helper).  
    measurements : list de longitud n_trials
        Cada elemento es el output de `collect_other_measurements(env, done, reward)`,
        es decir, o bien `[ [juice_L, juice_R], (nB, nA), chosen_juice ]` para trials
        exitosos, o `None` para trials abortados/sin recompensa (ver :contentReference[oaicite:1]{index=1}).  
    window_size : int
        Número de valores no-NaN, contando desde el final de cada trial válido, usados
        para promediar la FR en la “decision window”.  
    R_A, R_B : float
        Recompensa por unidad de Jugo A y Jugo B, respectivamente.  
    p_global_thresh : float, opcional (default=0.01)
        Umbral de significancia para el F-test global del modelo.  
    p_individual_thresh : float, opcional (default=0.05)
        Umbral de significancia individual para βi. En esta versión, para simplificar,
        no requerimos explícitamente p_i < p_individual_thresh; en cambio, elegimos
        siempre el β con menor p-valor entre los β₁..β₆, si el modelo global fue
        significativo.  

    Retorna:
    --------
    classifications : list de longitud n_neurons
        Cada entrada es el tipo de neurona:
          - "non-selective"
          - "action-value-right"
          - "action-value-left"
          - "chosen-value-difference"
          - "chosen-value-sum"
          - "chosen-action"
          - "juice-position"
    """

    n_neurons, n_timesteps, n_trials = FR_tensor.shape

    if len(measurements) != n_trials:
        raise ValueError(f"measurements debe tener longitud igual a n_trials ({n_trials}), "
                         f"pero obtuvo {len(measurements)}")

    # 1) Identificar únicamente los trials “válidos” (measurement != None):
    valid_trials = [t for t in range(n_trials) if measurements[t] is not None]
    if len(valid_trials) == 0:
        raise ValueError("No hay ningún trial válido (todos measurements[t] son None).")

    # 2) Para cada trial válido t, construir los regresores:
    #    Adiff, AVR, AVL, CV_diff, CV_sum, Jpos.
    #    Los almacenaremos en un array X_full de shape (n_valid, 6)
    n_valid = len(valid_trials)
    X_full = np.zeros((n_valid, 6))  # columnas = [Adiff, AVR, AVL, CV_diff, CV_sum, Jpos]

    for idx, t in enumerate(valid_trials):
        juice_pair_LR, offer_pair_BA, chosen_juice = measurements[t]
        juice_L, juice_R = juice_pair_LR
        nB, nA = offer_pair_BA

        # 2.a) Jpos = 1 si 'A' está a la izquierda, 0 si 'A' está a la derecha
        Jpos = 1 if juice_L == 'A' else 0

        # 2.b) Valores teóricos AVL, AVR según qué jugo está a la izquierda (ver env.trial_rL, trial_rR)
        if juice_L == 'A':
            AVL = nA * R_A
            AVR = nB * R_B
        else:
            AVL = nB * R_B
            AVR = nA * R_A

        # 2.c) Deducir Adiff. Si chosen_juice coincide con juice_L → acción Left (–1). Si coincide con juice_R → Right (+1).
        if chosen_juice == juice_L:
            Adiff = -1
        elif chosen_juice == juice_R:
            Adiff = +1
        else:
            # ¡No debería ocurrir porque collect_other_measurements garantiza consistencia! 
            raise ValueError(f"Trial {t}: chosen_juice (‘{chosen_juice}’) no coincide con juice_L (‘{juice_L}’) ni juice_R (‘{juice_R}’).")

        # 2.d) CVL, CVR (chosen value left/right)
        if Adiff == -1:
            CVL = AVL
            CVR = 0.0
        else:
            CVR = AVR
            CVL = 0.0

        # 2.e) Llenar la fila idx de X_full:
        X_full[idx, 0] = Adiff
        X_full[idx, 1] = AVR
        X_full[idx, 2] = AVL
        X_full[idx, 3] = CVR - CVL       # CV_diff
        X_full[idx, 4] = CVR + CVL       # CV_sum
        X_full[idx, 5] = Jpos

    # 3) Construir FR_avg sólo para los trials válidos:
    #    FR_avg tiene shape (n_neurons, n_valid), donde FR_avg[m, idx]
    #    = media de los últimos window_size timesteps no-NaN de FR_tensor[m, :, t=valid_trials[idx]].
    FR_avg = np.zeros((n_neurons, n_valid))

    for idx, t in enumerate(valid_trials):
        # Hallar los índices no-NaN para este trial t (por convención, basta chequear neurona 0):
        valid_indices = np.where(~np.isnan(FR_tensor[0, :, t]))[0]
        if valid_indices.size == 0:
            # Si no hay valores válidos, ponemos NaN en FR_avg, y luego
            # al ajustar ese trial se eliminará de forma implícita.
            FR_avg[:, idx] = np.nan
            continue

        # Tomar los últimos window_size índices:
        if valid_indices.size >= window_size:
            last_inds = valid_indices[-window_size:]
        else:
            last_inds = valid_indices  # si hay menos de window_size, tomamos todos

        # Calcular la media sobre esos índices para cada neurona m
        FR_avg[:, idx] = np.mean(FR_tensor[:, last_inds, t], axis=1)

    # 4) Añadir intercepto a X_full:
    X_with_const = sm.add_constant(X_full, prepend=True)  # shape = (n_valid, 7)

    # 5) Para cada neurona m, ajustar OLS sobre los trials válidos con FR_avg[m, :]
    classifications = []
    tipo_por_reg_index = {
        0: "chosen-action",            # β1 = Adiff
        1: "action-value-right",       # β2 = AVR
        2: "action-value-left",        # β3 = AVL
        3: "chosen-value-difference",  # β4 = CVR − CVL
        4: "chosen-value-sum",         # β5 = CVR + CVL
        5: "juice-position"            # β6 = Jpos
    }

    for m in range(n_neurons):
        Y_m_all = FR_avg[m, :]  # shape (n_valid,)

        # 5.a) Filtrar trials que, a pesar de ser válidos, 
        #      hayan quedado con FR_avg[m, idx] = NaN (por ausencia total de spikes)
        mask_valid_Y = ~np.isnan(Y_m_all)
        if np.sum(mask_valid_Y) < 5:
            # Si quedan menos de 5 puntos válidos para ajustar, lo consideramos “non-selective”
            classifications.append("non-selective")
            continue

        Y_valid = Y_m_all[mask_valid_Y]                # (n_valid_effectivos,)
        X_valid = X_with_const[mask_valid_Y, :]         # (n_valid_effectivos, 7)

        # 5.b) Ajuste OLS
        model = sm.OLS(Y_valid, X_valid).fit()

        if model.f_pvalue < p_global_thresh:
            # Es “value-encoding”: hallamos el β₁..β₆ con menor p-valor
            pvals = model.pvalues[1:]    # descartamos el p-valor del intercept (posición 0)
            idx_min_p = int(np.argmin(pvals))  # índice en 0..5
            neuron_type = tipo_por_reg_index[idx_min_p]
        else:
            neuron_type = "non-selective"

        classifications.append(neuron_type)

    return classifications

In [3]:
# --- RNN EconomicChoice Partial ---
pkl_path = "outputs/rnn_nohold_partial_min.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_1, \
    actor_loss_history_1, \
    critic_loss_history_1, \
    actor_firing_rates_1, \
    critic_firing_rates_1, \
    measurements_juices_1 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_2.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_2, \
    actor_loss_history_2, \
    critic_loss_history_2, \
    actor_firing_rates_2, \
    critic_firing_rates_2, \
    measurements_juices_2 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_3.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_3, \
    actor_loss_history_3, \
    critic_loss_history_3, \
    actor_firing_rates_3, \
    critic_firing_rates_3, \
    measurements_juices_3 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_4.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_4, \
    actor_loss_history_4, \
    critic_loss_history_4, \
    actor_firing_rates_4, \
    critic_firing_rates_4, \
    measurements_juices_4 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_5.pkl"
with open(pkl_path, "rb") as f: 
    total_rewards_history_5, \
    actor_loss_history_5, \
    critic_loss_history_5, \
    actor_firing_rates_5, \
    critic_firing_rates_5, \
    measurements_juices_5 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_6.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_6, \
    actor_loss_history_6, \
    critic_loss_history_6, \
    actor_firing_rates_6, \
    critic_firing_rates_6, \
    measurements_juices_6 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_7.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_7, \
    actor_loss_history_7, \
    critic_loss_history_7, \
    actor_firing_rates_7, \
    critic_firing_rates_7, \
    measurements_juices_7 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_8.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_8, \
    actor_loss_history_8, \
    critic_loss_history_8, \
    actor_firing_rates_8, \
    critic_firing_rates_8, \
    measurements_juices_8 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_9.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_9, \
    actor_loss_history_9, \
    critic_loss_history_9, \
    actor_firing_rates_9, \
    critic_firing_rates_9, \
    measurements_juices_9 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_10.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_10, \
    actor_loss_history_10, \
    critic_loss_history_10, \
    actor_firing_rates_10, \
    critic_firing_rates_10, \
    measurements_juices_10 = pickle.load(f)

pkl_path = "outputs/rnn_nohold_partial_min_stage_11.pkl"
with open(pkl_path, "rb") as f:
    total_rewards_history_11, \
    actor_loss_history_11, \
    critic_loss_history_11, \
    actor_firing_rates_11, \
    critic_firing_rates_11, \
    measurements_juices_11 = pickle.load(f)

total_rewards_history_EC_rnn_P = np.concatenate([total_rewards_history_1, total_rewards_history_2, total_rewards_history_3, total_rewards_history_4, total_rewards_history_5, total_rewards_history_6, total_rewards_history_7, total_rewards_history_8, total_rewards_history_9, total_rewards_history_10, total_rewards_history_11])
actor_loss_history_EC_rnn_P = np.concatenate([actor_loss_history_1, actor_loss_history_2, actor_loss_history_3, actor_loss_history_4, actor_loss_history_5, actor_loss_history_6, actor_loss_history_7, actor_loss_history_8, actor_loss_history_9, actor_loss_history_10, actor_loss_history_11])
critic_loss_history_EC_rnn_P = np.concatenate([critic_loss_history_1, critic_loss_history_2, critic_loss_history_3, critic_loss_history_4, critic_loss_history_5, critic_loss_history_6, critic_loss_history_7, critic_loss_history_8, critic_loss_history_9, critic_loss_history_10, critic_loss_history_11])

In [12]:
classify_neurons_by_last_valid(actor_firing_rates_11, measurements_juices_11, 100, 2.2, 1.0)

/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-package

['juice-position',
 'non-selective',
 'juice-position',
 'juice-position',
 'non-selective',
 'action-value-right',
 'non-selective',
 'juice-position',
 'non-selective',
 'chosen-value-sum',
 'non-selective',
 'non-selective',
 'non-selective',
 'juice-position',
 'juice-position',
 'juice-position',
 'non-selective',
 'non-selective',
 'non-selective',
 'juice-position',
 'non-selective',
 'non-selective',
 'chosen-value-difference',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'juice-position',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'juice-position',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'non-selective',
 'juice-position',
 'juice-position']

In [ ]:
def classify_neurons_with_stats(
    FR_tensor: np.ndarray,
    measurements: list,
    window_size: int,
    R_A: float,
    R_B: float,
    p_global_thresh: float = 0.01
) -> tuple:
    """
    Clasifica cada neurona usando solamente los trials válidos (measurements[t] != None).
    Para cada neurona m y cada trial válido t:
      - Promedia los últimos `window_size` timesteps no-NaN de FR_tensor[m, :, t] como FR_m(t).
      - Ajusta la regresión:
            FR_m(t) = β0
                   + β1·Adiff
                   + β2·AVR
                   + β3·AVL
                   + β4·(CVR − CVL)
                   + β5·(CVR + CVL)
                   + β6·Jpos
                   + ε
      - Si el F-test global p < p_global_thresh, la neurona es “value‐encoding” y se le asigna un tipo
        según el coeficiente β₁..β₆ con p-valor más pequeño.
      - Si no, se clasifica como “non-selective”.

    Devuelve tres objetos:
      1) classifications: lista de longitud n_neurons con la etiqueta de cada neurona.
      2) coefs_df: DataFrame con filas correspondientes a neuronas (índice = número de neurona),
         columnas = ["Adiff", "AVR", "AVL", "CV_diff", "CV_sum", "Jpos"], conteniendo β₁..β₆.
         Se eliminan de este DataFrame aquellas filas (neuronas) cuyos 6 p-valores sean todos NaN.
      3) pvals_df: DataFrame idéntico en índice y columnas, conteniendo los p-valores de β₁..β₆,
         también sin las filas donde todos los p-valores son NaN.

    Parámetros:
    -----------
    FR_tensor : np.ndarray, shape = (n_neurons, n_timesteps, n_trials)
        Firing‐rates de cada neurona en cada timestep de cada trial. Los timesteps
        “extra” tras la finalización de un trial están rellenados con NaN para todos 
        los canales.
    measurements : list de longitud n_trials
        Output de collect_other_measurements. Cada elemento es o bien
        [ [juice_L, juice_R], (nB, nA), chosen_juice ] o None.
    window_size : int
        Número de valores no-NaN al final de cada trial para promediar FR.
    R_A, R_B : float
        Recompensa por unidad de Jugo A y Jugo B.
    p_global_thresh : float
        Umbral para el F-test global.

    Retorna:
    --------
    classifications : list de str, longitud = n_neurons
        Etiqueta de cada neurona:
          - "non-selective"
          - "action-value-right"
          - "action-value-left"
          - "chosen-value-difference"
          - "chosen-value-sum"
          - "chosen-action"
          - "juice-position"

    coefs_df : pandas.DataFrame, shape <= (n_neurons, 6)
        Valores de β₁..β₆ para cada neurona. Columnas:
          ["Adiff", "AVR", "AVL", "CV_diff", "CV_sum", "Jpos"].
        Filas donde todos los p-valores eran NaN han sido eliminadas.

    pvals_df : pandas.DataFrame, shape <= (n_neurons, 6)
        p-valores de β₁..β₆ para cada neurona. Mismo formato que coefs_df.
        También sin las filas donde todos los p-valores son NaN.
    """

    n_neurons, n_timesteps, n_trials = FR_tensor.shape

    if len(measurements) != n_trials:
        raise ValueError(f"measurements debe tener longitud = n_trials ({n_trials}), "
                         f"pero obtuvo {len(measurements)}")

    # 1) Identificar trials válidos (measurements[t] != None)
    valid_trials = [t for t in range(n_trials) if measurements[t] is not None]
    n_valid = len(valid_trials)
    if n_valid == 0:
        raise ValueError("No hay ningún trial válido (todos measurements[t] son None).")

    # 2) Construir matriz de regresores X_full (n_valid x 6)
    X_full = np.zeros((n_valid, 6))
    for idx, t in enumerate(valid_trials):
        juice_pair_LR, offer_pair_BA, chosen_juice = measurements[t]
        juice_L, juice_R = juice_pair_LR
        nB, nA = offer_pair_BA

        # Jpos: 1 si 'A' está a la izquierda, 0 si 'A' está a la derecha
        Jpos = 1 if juice_L == 'A' else 0

        # Valores AVL, AVR según qué zumo está a la izquierda
        if juice_L == 'A':
            AVL = nA * R_A
            AVR = nB * R_B
        else:
            AVL = nB * R_B
            AVR = nA * R_A

        # Adiff: -1 si eligió izquierda, +1 si eligió derecha
        if chosen_juice == juice_L:
            Adiff = -1
        elif chosen_juice == juice_R:
            Adiff = +1
        else:
            raise ValueError(
                f"Trial {t}: chosen_juice (‘{chosen_juice}’) no coincide con juice_L (‘{juice_L}’) ni juice_R (‘{juice_R}’)."
            )

        # CVL, CVR
        if Adiff == -1:
            CVL = AVL
            CVR = 0.0
        else:
            CVR = AVR
            CVL = 0.0

        # Llenar X_full[idx]
        X_full[idx, 0] = Adiff
        X_full[idx, 1] = AVR
        X_full[idx, 2] = AVL
        X_full[idx, 3] = CVR - CVL    # CV_diff
        X_full[idx, 4] = CVR + CVL    # CV_sum
        X_full[idx, 5] = Jpos

    # 3) Construir FR_avg (n_neurons x n_valid)
    FR_avg = np.zeros((n_neurons, n_valid))
    for idx, t in enumerate(valid_trials):
        # Encontrar índices no-NaN para este trial t
        valid_indices = np.where(~np.isnan(FR_tensor[0, :, t]))[0]
        if valid_indices.size == 0:
            FR_avg[:, idx] = np.nan
            continue
        # Tomar los últimos window_size índices
        if valid_indices.size >= window_size:
            last_inds = valid_indices[-window_size:]
        else:
            last_inds = valid_indices
        # Promedio de FR en esos índices para cada neurona
        FR_avg[:, idx] = np.mean(FR_tensor[:, last_inds, t], axis=1)

    # 4) Agregar intercepto a X_full
    X_with_const = sm.add_constant(X_full, prepend=True)  # (n_valid, 7)

    # 5) Preparar contenedores para coeficientes y p-valores
    reg_names = ["Adiff", "AVR", "AVL", "CV_diff", "CV_sum", "Jpos"]
    coefs_arr = np.full((n_neurons, 6), np.nan, dtype=float)
    pvals_arr = np.full((n_neurons, 6), np.nan, dtype=float)
    classifications = []

    tipo_por_reg_index = {
        0: "chosen-action",
        1: "action-value-right",
        2: "action-value-left",
        3: "chosen-value-difference",
        4: "chosen-value-sum",
        5: "juice-position"
    }

    # 6) Ajustar regresión para cada neurona
    for m in range(n_neurons):
        Y_all = FR_avg[m, :]  # (n_valid,)

        # Filtrar trials donde FR_avg sea NaN
        valid_mask = ~np.isnan(Y_all)
        n_effective = valid_mask.sum()
        if n_effective < 5:
            classifications.append("non-selective")
            continue

        Y = Y_all[valid_mask]
        X = X_with_const[valid_mask, :]

        model = sm.OLS(Y, X).fit()

        # Extraer β₁..β₆ y sus p-valores (model.params[1:] y model.pvalues[1:])
        betas = model.params[1:]   # longitud 6
        pvals = model.pvalues[1:]  # longitud 6

        coefs_arr[m, :] = betas
        pvals_arr[m, :] = pvals

        # Clasificación según F-test global
        if model.f_pvalue < p_global_thresh:
            idx_min_p = int(np.argmin(pvals))
            classifications.append(tipo_por_reg_index[idx_min_p])
        else:
            classifications.append("non-selective")

    # 7) Convertir coefs_arr y pvals_arr a DataFrames con índice = número de neurona
    coefs_df = pd.DataFrame(coefs_arr, columns=reg_names, index=np.arange(n_neurons))
    pvals_df = pd.DataFrame(pvals_arr, columns=reg_names, index=np.arange(n_neurons))

    # 8) Eliminar filas donde todos los p-valores (las 6 columnas) sean NaN
    mask_some_pval = ~pvals_df.isna().all(axis=1)
    coefs_df = coefs_df.loc[mask_some_pval]
    pvals_df = pvals_df.loc[mask_some_pval]

    return classifications, coefs_df, pvals_df

In [17]:
classifications, coefs_df, pvals_df = classify_neurons_with_stats(
    actor_firing_rates_11,
    measurements_juices_11,
    100,
    2.2,
    1.0,
    p_global_thresh=0.01
)

# 6) Imprimimos resultados
for m, t in enumerate(classifications):
    print(f"Neurona {m:03d}: {t}")

print("\nCoeficientes β₁..β₆ por neurona:")
print(coefs_df)

print("\nP-valores de β₁..β₆ por neurona:")
print(pvals_df)

Neurona 000: juice-position
Neurona 001: non-selective
Neurona 002: juice-position
Neurona 003: juice-position
Neurona 004: non-selective
Neurona 005: action-value-right
Neurona 006: non-selective
Neurona 007: juice-position
Neurona 008: non-selective
Neurona 009: chosen-value-sum
Neurona 010: non-selective
Neurona 011: non-selective
Neurona 012: non-selective
Neurona 013: juice-position
Neurona 014: juice-position
Neurona 015: juice-position
Neurona 016: non-selective
Neurona 017: non-selective
Neurona 018: non-selective
Neurona 019: juice-position
Neurona 020: non-selective
Neurona 021: non-selective
Neurona 022: chosen-value-difference
Neurona 023: non-selective
Neurona 024: non-selective
Neurona 025: non-selective
Neurona 026: non-selective
Neurona 027: non-selective
Neurona 028: non-selective
Neurona 029: non-selective
Neurona 030: non-selective
Neurona 031: non-selective
Neurona 032: non-selective
Neurona 033: juice-position
Neurona 034: non-selective
Neurona 035: non-selective
N

/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/mnt/c/Users/imcir/Desktop/CIENCIA DATOS/cienciadatosenv/lib/python3.12/site-package